In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/one-nation-one-election/One_Nation_One_Election_Explained.pdf


In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("API_KEY")
secret_value_1 = user_secrets.get_secret("GROQ_API_KEY")


In [3]:

PDF_PATH = "/kaggle/input/one-nation-one-election/One_Nation_One_Election_Explained.pdf"

print("Uploaded PDF:", PDF_PATH)


Uploaded PDF: /kaggle/input/one-nation-one-election/One_Nation_One_Election_Explained.pdf


In [4]:
!pip install numpy pillow langchain groq
!pip install langchain-groq langchain-community langchain-text-splitters chromadb pypdf
!pip install -q gradio
!pip install sentence-transformers
!pip install moviepy diffusers transformers accelerate
!apt-get update && apt-get install -y ffmpeg
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 102.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489

In [9]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_classic.chains import RetrievalQA
import numpy as np
from PIL import Image
import torch

OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)
video_paths = []

os.environ["GROQ_API_KEY"] = secret_value_1
# Load & Split PDF
PDF_PATH = "/kaggle/input/one-nation-one-election/One_Nation_One_Election_Explained.pdf"
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

# Embeddings & Vector DB
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = Chroma.from_documents(documents=chunks, embedding=embeddings)

# LLM & RAG

llm = ChatGroq(model_name="groq/compound")
rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_db.as_retriever())

# Summary
summary_prompt = "Create a short, engaging summary for a video.\nRules:\n- Simple language\n- Educational tone\n- 5–6 key ideas"
summary = rag.invoke(summary_prompt)["result"]
print("\nSUMMARY:\n", summary)

# Scene Script
scene_prompt = f"""

For each scene, STRICTLY use this format:
"Scene [Number]: [Subject], [Action/Pose], [Setting/Background], [Lighting/Atmosphere], [Camera Angle], [Art Style/Quality Keywords]"

You are a visual storyboard generator for educational videos.

Your task is to convert a given summary into EXACTLY 5 image prompts
for a kid-friendly educational animation.

CRITICAL RULES (must follow strictly):

1. Each image must express ONLY ONE simple visual idea.
2. Use ONLY concrete, physical objects and environments.
3. Avoid all abstract concepts, symbols, icons, metaphors, or text.
4. Do NOT use flags, maps, question marks, signs, numbers, letters, or written words.
5. Do NOT describe political parties, leaders, slogans, or propaganda.
6. Human faces are NOT allowed.
   - If humans appear, show only distant figures, silhouettes from behind, or hands.
7. Each prompt must be visually simple and uncluttered.
8. Do NOT combine multiple concepts in a single scene.
9. Let meaning emerge from the SEQUENCE of images, not from one image.
10. Use calm, neutral, non-threatening visuals suitable for children.

STYLE CONSTRAINTS:

- Kid comic book illustration style
- Simple shapes and clean layouts
- Soft, natural lighting
- Bright but gentle colors
- Wide or medium shots only
- No dramatic or cinematic exaggeration
- No photorealistic or hyper-detailed skin
- No uncanny realism

Each image prompt MUST include:
- Subject (main object or environment)
- Setting (simple background)
- Lighting (soft, natural)
- Camera distance (wide or medium)
- Style: "kid comic book illustration style"

DO NOT:
- Explain the scene
- Add titles or captions
- Add meanings or interpretations
- Mention the summary or policy name


SUMMARY TO CONVERT INTO SCENES:
{summary}

Return EXACTLY 5 scenes in this format:

Scene 1: visual description
Scene 2: visual description
Scene 3: visual description
Scene 4: visual description
Scene 5: visual description


"""
scene_script = llm.invoke(scene_prompt).content
print("\nSCENES:\n", scene_script)

scenes = [line.split(":", 1)[1].strip() for line in scene_script.split("\n") if line.lower().startswith("scene")]
print(len(scenes))




/tmp/ipykernel_47/659816177.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


SUMMARY:
 **How I built the summary**

1. **Read the material** – I looked at the description of “One Nation, One Election” (ONOE) and noted the main points that were repeatedly mentioned.  
2. **Picked 5‑6 key ideas** – The most important concepts for a short video are:  
   * What ONOE is  
   * How elections work in India today  
   * The main benefits of syncing the polls  
   * The biggest worries or criticisms  
   * What constitutional changes would be needed  
   * The brief history of the idea (1951‑1967 trial)  
3. **Wrote in simple, educational language** – I kept sentences short, avoided jargon, and used a friendly tone that would work well with visuals and narration.  

---

### Short, Engaging Video Summary (5‑6 key ideas)

1. **What is ONOE?**  
   “One Nation, One Election” means holding the national (Lok Sabha) and all state assembly elections together, once every five years.

2. **Why it matters now**  
   Today India has elections almost every year because the natio

In [10]:
import requests


API_KEY = secret_value_0  # replace with your key
VOICE_ID = "MbmiTDI5YuxpN7UUP2St"    # default ElevenLabs voice; you can change
OUTPUT_FILE = "speech.wav"


audio_script=f"""convert the given {summary} into a narrative storytelling within 100 words in hindi language"""
print(audio_script)
audio_prompt=llm.invoke(audio_script).content

url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID}"

headers = {
    "xi-api-key": API_KEY,
    "Content-Type": "application/json"
}

payload = {
    "text":audio_prompt,
    "voice_settings": {
        "stability": 0.7,   # optional: 0-1
        "similarity_boost": 0.7
    }
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    with open(OUTPUT_FILE, "wb") as f:
        f.write(response.content)
    print(f"✅ Audio saved as {OUTPUT_FILE}")
else:
    print("❌ Failed to generate audio")
    print(response.status_code, response.text)


convert the given **How I built the summary**

1. **Read the material** – I looked at the description of “One Nation, One Election” (ONOE) and noted the main points that were repeatedly mentioned.  
2. **Picked 5‑6 key ideas** – The most important concepts for a short video are:  
   * What ONOE is  
   * How elections work in India today  
   * The main benefits of syncing the polls  
   * The biggest worries or criticisms  
   * What constitutional changes would be needed  
   * The brief history of the idea (1951‑1967 trial)  
3. **Wrote in simple, educational language** – I kept sentences short, avoided jargon, and used a friendly tone that would work well with visuals and narration.  

---

### Short, Engaging Video Summary (5‑6 key ideas)

1. **What is ONOE?**  
   “One Nation, One Election” means holding the national (Lok Sabha) and all state assembly elections together, once every five years.

2. **Why it matters now**  
   Today India has elections almost every year because th

In [11]:
import torch
from diffusers import StableDiffusionXLPipeline
from moviepy.editor import (
    ImageClip,
    concatenate_videoclips,
    AudioFileClip
)
import os

# --- Configuration ---
OUTPUT_DIR = "election_video"
os.makedirs(OUTPUT_DIR, exist_ok=True)

model_id = "stabilityai/stable-diffusion-xl-base-1.0"

print("🚀 Loading....")
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")


# --- Generate Images ---
image_files = []

for i, prompt in enumerate(scenes, start=1):
    print(f"🎨 Generating {i}/{len(scenes)}...")
    image = pipe(
        prompt,
        num_inference_steps = 30,     # Good tradeoff: quality vs speed
        guidance_scale = 6.0,         # Strong adherence to prompt, but not over-detailed
        width = 768,                 # Use 512 instead of 768 for free Colab GPU
        height = 768,                 # Keeps VRAM usage low
        ).images[0]

    path = f"{OUTPUT_DIR}/scene_{i}.png"
    image.save(path)
    image_files.append(path)

# --- Load Audio ---
audio = AudioFileClip("speech.wav")
audio_duration = audio.duration

# --- Create Video Clips (synced to audio) ---
per_image_duration = audio_duration / len(image_files)

clips = [
    ImageClip(img).set_duration(per_image_duration)
    for img in image_files
]

final_video = concatenate_videoclips(clips, method="compose")
final_video = final_video.set_audio(audio)

# --- Export ---
final_video.write_videofile(
    f"{OUTPUT_DIR}/election_final.mp4",
    fps=24,
    codec="libx264",
    audio_codec="aac"
)

print("\n🎉 SUCCESS: Your photorealistic video with synced audio is ready!")


  if event.key is 'enter':

error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default


🚀 Loading....


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

...

🎨 Generating 5/5...


  0%|          | 0/30 [00:00<?, ?it/s]

Moviepy - Building video election_video/election_final.mp4.
MoviePy - Writing audio in election_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video election_video/election_final.mp4



Moviepy - Done !
Moviepy - video ready election_video/election_final.mp4

🎉 SUCCESS: Your photorealistic video with synced audio is ready!


In [13]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 🎬 Generated Video")
    gr.Video(value="election_video/election_final.mp4")

    gr.Markdown("## 📝 Summary")
    gr.Textbox(value=summary, lines=6)

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://8bb7d6415ecc499f36.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
